Bilgisayar Mühendisliği <br>
Eda İpek Sanlı 20040101006 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA ##################################################
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.discriminant_analysis import StandardScaler
from sklearn.tree import DecisionTreeClassifier



In [3]:
#Dosyayı okur ve data frame'e çevirir
#delimiter sütunları ayıran karakteri belirtirken, header veri setinin başlık satırı olmadığını belirtir.
df = pd.read_csv('Pima Indians Diabetes.txt', header=None, delimiter='\t')
column_names = ['Pregnencies', 'Glucose', 'BloodPressure','SkinThickness','SerumInsulin','BMI','DiabetesPedigreeFunction','Age','Output'] 

df.columns = column_names

df.head(10)

,Pregnencies,Glucose,BloodPressure,SkinThickness,SerumInsulin,BMI,DiabetesPedigreeFunction,Age,Output
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [4]:
row,col=df.shape
print('Number of rows:',row)
print('Number of columns:',col)

Number of rows: 768
Number of columns: 9


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnencies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   SerumInsulin              768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Output                    768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnencies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
SerumInsulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Output,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [7]:
#Eksik verinin var olup olmadığını kontrol eder
df.isnull().sum()

Pregnencies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
SerumInsulin                0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Output                      0
dtype: int64

In [8]:
X = df.drop('Output',axis=1)
y = df['Output']        

sc = StandardScaler()
X_normalized = sc.fit_transform(X)

print("Normalleştirilmiş Veri:")
print(X_normalized)


Normalleştirilmiş Veri:
[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [18]:
#PCA Uygulaması
pca = PCA(n_components=2)  # İki en yüksek öz değeri almak için
X_pca = pca.fit_transform(X_normalized)

pca_df = pd.DataFrame(data=X_pca, columns=['PCA_Component1', 'PCA_Component2'])
pca_df = pd.concat([pca_df, df['Output']], axis=1)

print("PCA'nın oluşturduğu iki ana bileşen: ")
print(pca.components_)

# En önemli iki öznitelik için isimleri bulma
most_important = [np.abs(pca.components_[i]).argmax() for i in range(len(pca.components_))]
most_important_names = [df.columns[most_important[i]] for i in range(len(pca.components_))]

print("En önemli iki öznitelik: ", most_important_names)

pca_df


PCA'nın oluşturduğu iki ana bileşen: 
[[ 0.1284321   0.39308257  0.36000261  0.43982428  0.43502617  0.45194134
   0.27061144  0.19802707]
 [ 0.59378583  0.17402908  0.18389207 -0.33196534 -0.25078106 -0.1009598
  -0.122069    0.62058853]]
En önemli iki öznitelik:  ['BMI', 'Age']


,PCA_Component1,PCA_Component2,Output
0,1.068503,1.234895,1
1,-1.121683,-0.733852,0
2,-0.396477,1.595876,1
3,-1.115781,-1.271241,0
4,2.359334,-2.184819,1
...,...,...,...
763,1.562085,1.923150,0
764,-0.100405,-0.614181,0
765,-0.283475,0.097065,0
766,-1.060324,0.837062,1


In [23]:
#LDA uygulaması

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X, y)

print("LDA sonuçları: ", X_lda)

most_important_index = np.argmax(np.abs(lda.coef_))

# En önemli öznitelik için ismi bulma
most_important_name = df.columns[most_important_index]

print("En önemli öznitelik: ", most_important_name)

LDA sonuçları:  [[ 1.20734465e+00]
 [-1.04667925e+00]
 [ 3.58137789e-01]
 [-1.25358639e+00]
 [ 7.38356773e-01]
 [-6.43181548e-01]
 [-1.06395989e+00]
 [-1.11817360e+00]
 [ 2.13559777e+00]
 [ 1.40324022e-02]
 [-1.06051075e-01]
 [ 1.05998781e+00]
 [ 1.59421698e+00]
 [ 2.35533764e+00]
 [ 1.18273903e+00]
 [-1.49982562e+00]
 [ 8.07026270e-01]
 [-3.58487133e-01]
 [-6.50790173e-01]
 [-1.19022629e-01]
 [ 9.01008777e-01]
 [ 6.76855270e-01]
 [ 1.57945959e+00]
 [ 3.41198687e-01]
 [ 2.19754008e+00]
 [ 8.69577152e-01]
 [ 8.10083168e-01]
 [-1.17836287e+00]
 [ 1.70736316e+00]
 [ 2.60512508e-01]
 [ 1.02820941e+00]
 [ 8.36509126e-01]
 [-1.43697679e+00]
 [-7.67464615e-01]
 [ 9.61242817e-01]
 [-7.55045862e-02]
 [ 6.96154758e-01]
 [ 9.60895532e-01]
 [-4.05136298e-01]
 [ 1.73049289e+00]
 [ 2.37945417e-01]
 [ 8.07669789e-01]
 [ 4.18876820e-01]
 [ 3.21102140e+00]
 [ 2.53278379e-01]
 [ 9.13371988e-01]
 [-7.48698763e-01]
 [-1.19969865e+00]
 [ 1.62282595e-01]
 [-2.79058556e+00]
 [-1.15440032e+00]
 [-1.39630249e+

In [10]:
#Ana veri setine Çoklu Doğrusal Regresyon ve Multinominal Lojistik Regresyon modeli
X = df.drop(columns=['Output'])
y = df['Output']

# Veri setini eğitim ve test alt kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Çoklu Doğrusal Regresyon modeli
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)
linear_reg_coefficients = linear_reg_model.coef_

# Multinominal Lojistik Regresyon modeli
logistic_reg_model = LogisticRegression(max_iter=1000)
logistic_reg_model.fit(X_train, y_train)
logistic_reg_coefficients = logistic_reg_model.coef_

# Katsayıları raporlama
print("Çoklu Doğrusal Regresyon katsayıları:")
print(linear_reg_coefficients)
print("\nMultinominal Lojistik Regresyon katsayıları:")
print(logistic_reg_coefficients)

linear_reg_predictions = linear_reg_model.predict(X_test)
logistic_reg_predictions = logistic_reg_model.predict(X_test)

Çoklu Doğrusal Regresyon katsayıları:
[ 0.00899394  0.00574161 -0.00171114 -0.00025321 -0.0001265   0.01617347
  0.07290169  0.0062645 ]

Multinominal Lojistik Regresyon katsayıları:
[[ 0.0577509   0.03590632 -0.01087037 -0.00140237 -0.00098556  0.10910568
   0.37409764  0.0360156 ]]


In [19]:
#PCA verilerine göre Çoklu Doğrusal Regresyon ve Multinominal Lojistik Regresyon modeli
X = pca_df.drop(columns=['Output'])
y = pca_df['Output']

# Veri setini eğitim ve test alt kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Çoklu Doğrusal Regresyon modeli
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)
linear_reg_coefficients = linear_reg_model.coef_

# Multinominal Lojistik Regresyon modeli
logistic_reg_model = LogisticRegression(max_iter=1000)
logistic_reg_model.fit(X_train, y_train)
logistic_reg_coefficients = logistic_reg_model.coef_

# Katsayıları raporlama
print("Çoklu Doğrusal Regresyon katsayıları:")
print(linear_reg_coefficients)
print("\nMultinominal Lojistik Regresyon katsayıları:")
print(logistic_reg_coefficients)

linear_reg_predictions = linear_reg_model.predict(X_test)
logistic_reg_predictions = logistic_reg_model.predict(X_test)

Çoklu Doğrusal Regresyon katsayıları:
[0.13965659 0.07903115]

Multinominal Lojistik Regresyon katsayıları:
[[0.76804592 0.40598983]]


In [12]:
#Ana veri seti için Naive Bayes metodu
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import confusion_matrix


# Veri setini %70 eğitim ve %30 test olacak şekilde ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Naive Bayes sınıflandırıcısını uygulama
nb = GaussianNB()
nb.fit(X_train, y_train)

# Eğitim seti için tahminler yapma
y_train_pred = nb.predict(X_train)

# Test seti için tahminler yapma
y_test_pred = nb.predict(X_test)

# Performans metriklerini hesaplama
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
specificity = tn / (tn + fp)
sensitivity = tp /( tp + fn)

conf_matx = confusion_matrix(y_test,y_test_pred)
# Sonuçları yazdırma
print(f"Test accuracy: {test_accuracy}")
print(f"F1 Score: {test_f1}")
print(f"Specifitiy: {specificity}")
print(f"Sensitivity: {sensitivity}")
print(f"Conf matrix: {conf_matx}")


Test accuracy: 0.7445887445887446
F1 Score: 0.6424242424242423
Specifitiy: 0.7880794701986755
Sensitivity: 0.6625
Conf matrix: [[119  32]
 [ 27  53]]


In [25]:
#PCA uygulanmış transformasyon öznitelik matrisi için Naive Bayes metodu
X_pca_x= pca_df.drop('Output',axis=1)
y_pca= pca_df['Output']

X_train, X_test, y_train, y_test = train_test_split(X_pca_x, y_pca, test_size=0.3, random_state=42)

# Naive Bayes sınıflandırıcısını uygulama
nb = GaussianNB()
nb.fit(X_train, y_train)

# Eğitim seti için tahminler yapma
y_train_pred = nb.predict(X_train)

# Test seti için tahminler yapma
y_test_pred = nb.predict(X_test)

# Performans metriklerini hesaplama
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
specificity = tn / (tn + fp)
sensitivity = tp /( tp + fn)

conf_matx = confusion_matrix(y_test,y_test_pred)
# Sonuçları yazdırma
print(f"Test accuracy: {test_accuracy}")
print(f"F1 Score: {test_f1}")
print(f"Specifitiy: {specificity}")
print(f"Sensitivity: {sensitivity}")
print(f"Conf matrix: {conf_matx}")


Test accuracy: 0.70995670995671
F1 Score: 0.5314685314685315
Specifitiy: 0.8344370860927153
Sensitivity: 0.475
Conf matrix: [[126  25]
 [ 42  38]]
